<a href="https://colab.research.google.com/github/jneey2000/ML/blob/main/3_anatomy_of_a_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- This chapter is designed to get you started with **using neural networks to solve real problems**.
- You’ll consolidate the knowledge you gained from our first practical example in previous chapter
- You’ll apply what you’ve learned to three new problems covering **the three most common use cases of neural networks**:
   1. binary classification (이진 분류): 영화 리뷰를 긍정 또는 부정으로 분류하기
   1. multiclass classification (다중 분류): 신문 기사를 토픽으로 분류하기
   1. scalar regression (회귀): 부동산 데이터를 바탕으로 주택 가격을 예측하기


# Anatomy of a neural network

As you saw in the previous chapters, training a neural network revolves around the following objects  
(신경망 훈련에는 다음 요소들이 관련되어 있음):
- **Layers**, which are combined into a network (or model)  
 <font color="orange">**네트워크(또는 모델)**을 구성하는 **층**</font>
- **The input data** and corresponding **targets**  
 <font color="orange">**입력데이터**와 그에 상응하는 **타깃**</font>
- **The loss function**, which defines the feedback signal used for learning  
<font color="orange">학습에 사용할 피드백 신호를 정의하는 **손실함수**</font>
- **The optimizer**, which determines how learning proceeds  
<font color="orange">학습 진행 방식을 결정하는 **옵티마이저**</font>

You can visualize their interaction as illustrated in figure 3.1:   
- 연속된 층으로 구성된 네트워크가 입력 데이터를 예측으로 매핑함
- 손실함수는 예측과 타깃을 비교하여 네트워크의 예측이 얼마나 잘 맞는 지를 측정하는 손실 값을 만듦
- 옵티마이저는 손실 값을 사용하여 네트워크 가중치를 업데이트함

<div>
<img src="https://drive.google.com/uc?export=view&id=1VDwJZjrFqGHFkVqsMPiU2qA1uukeGrbA" width="500"/>
</div>

Let’s take a closer look at layers, networks, loss functions, and optimizers.

## Layers: the building blocks of deep learning

- **신경망의 핵심적인 데이터 구조는 층 (Layer)임**
- **A layer** is a data-processing module that takes as input one or more tensors and that outputs one or more tensors.  
 **층(layer)은 하나 이상의 텐서를 입력으로 받아 하나 이상의 텐서를 출력하는 데이터 처리 모듈임**  
-  어떤 종류의 층(layer)은 상태(state)가 없지만 대부분의 경우 **가중치라는 층의 상태**를 가짐  
가중치는 SGD(확률적 경사 하강법)에 의해 학습되는 하나 이상의 텐서이며  
여기에는 **네트워크가 학습한 지식(knowledge)이 담겨 있음**

@ Examples of Stateless layers (No weigths) : **Flatten, Pooling, and Dropout layers**

- Different layers are appropriate for different tensor formats and different types of data processing.  
 **<font color="orange">층마다 적절한 텐서 포맷과 데이터 처리 방식이 다름</font>**
- For instance, **simple vector data**, stored in **2D tensors of shape (samples, features)**, is often processed by densely connected layers, also called **fully connected** or **dense layers** (the Dense class in Keras).    
예를 들어, (samples, features)크기의 2D텐서가 저장된 간단한 벡터 데이터는 완전 연결 층(fully connected layer)이나 밀집 층(dense layer)이라고도 불리는 밀집 연결 층(densely connected layer)에 의해 처리되는 경우가 많음(케라스에서는 Dense 클래스임)
- **Sequence data**, stored in **3D tensors of shape (samples, timesteps, features)**, is typically processed by **recurrent layers** such as an **LSTM layer**.  
(samples, timesteps, features)크기의 3D 텐서에 저장된 시퀀스 데이터는 일반적으로 LSTM 레이어와 같은 순환 레이어에 의해 처리됨
- Image data, stored in **4D tensors** (samples, channels, height, width), is usually processed by **2D convolution layers (Conv2D)**.  
4D 텐서에 저장된 이미지 데이터는 일반적으로 2D 컨볼루션 레이어(Conv2D)에 의해 처리됨

- You can think of layers as the LEGO bricks of deep learning,  
a metaphor that is made explicit by frameworks like Keras.
- Building deep-learning models in Keras is done by **clipping together compatible layers to form useful data-transformation pipelines**  
(<font color="orange">**케라스에서는 호환 가능한 층들을 엮어 데이터 변환 pipline을 구성**</font>함).
- The notion of **layer compatibility** here refers specifically to the fact that every layer will **only accept input tensors of a certain shape and will return output tensors of a certain shape**.  
(여기에서 **층 호환성(layer compatibility)**은 <U>각 층이 특정 크기의 입력 텐서만 받고  
 특정 크기의 출력 텐서를 반환</U>한다는 사실을 말함.)

Consider the following example:

In [ ]:
from keras import layers

layer = layers.Dense(32, input_shape=(784,))

- We’re creating a layer that will only accept as input 2D tensors where the first dimension is 784 (**axis 0, the batch dimension, is unspecified, and thus any value would be accepted**).  
우리는 첫 번째 차원이 784인 2D 텐서만 허용하는 인풋 레이어를 생성하고 있음(배치 차원인 axis 0은 지정되지 않았으므로 모든 값이 허용됩니다).  

- This layer will return a tensor where the first dimension has been transformed to be 32.  
이 레이어는 첫 번째 차원이 32로 변환된 텐서를 반환함  

- Thus this layer can only be connected to a downstream layer that expects 32-dimensional vectors as its input.
따라서 이 레이어는 32차원 벡터를 입력으로 예상하는 다운스트림 레이어에만 연결될 수 있음  

- When using Keras, you don’t have to worry about compatibility, because the layers you add to your models are dynamically built to match the shape of the incoming layer.   
Keras를 사용하면 모델에 추가하는 레이어가 들어오는 레이어의 모양과 일치하도록 동적으로 구축되므로 호환성에 대해 걱정할 필요가 없음    

For instance, suppose you write the following:
예를 들어 다음과 같이 작성한다고 가정해 보겠음  

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, input_shape=(784,)))
model.add(layers.Dense(32))

**The second layer didn’t receive an input shape argument**—instead, it automatically inferred its input shape as being the output shape of the layer that came before.  
두 번째 층에는 input_shape 매개변수를 지정하지 않음  
그 대신 앞선 층의 출력 크기를 입력 크기로 자동으로 채택함

## Models: networks of layers


- Picking the right network architecture is more an art than a science (No theoretical proof)  
(올바른 네트워크 아키텍처를 선택하는 것은 과학이라기보다 예술임(이론적 증거 없음))
- Although there are some best practices and principles you can rely on, only practice can help you become a proper neural-network architect (Know-how).  
(신뢰할 수 있는 몇 가지 모범 사례와 원칙이 있지만 연습만이 올바른 신경망 설계자가 되는 데 도움이 될 수 있음(노하우))

The next few chapters will both teach you explicit principles for building neural networks and help you develop intuition as to what works or doesn’t work for specific problems.

## Loss functions and optimizers: keys to configuring the learning process

Once the network architecture is defined, you still have to choose two more things:

- **Loss function (objective function)**— The quantity that will be minimized during training. It represents a measure of success for the task at hand.  
손실 함수(목적 함수) - 훈련 중에 최소화되는 양. 이는 당면한 작업에 대한 성공의 척도를 나타냄.    
- **Optimizer**— Determines how the network will be updated based on the loss function. It implements a specific variant of stochastic gradient descent (SGD).  
옵티마이저 - 손실 함수를 기반으로 네트워크가 업데이트되는 방법을 결정함. SGD(확률적 경사하강법)의 특정 변형을 구현함.

### Multiloss networks:
- A neural network that has multiple outputs may have **multiple loss functions (one per output)**.   
여러 출력이 있는 신경망에는 여러 손실 함수(출력당 하나)가 있을 수 있음  
- But the gradient-descent process must be based on **a single scalar loss value**;
그러나 경사하강법 과정은 단일 스칼라 손실 값을 기반으로 해야 함
- So, for multiloss networks, all losses are combined (via averaging) into <font color="orange">**a single scalar quantity**</font>.  
따라서 다중 손실 네트워크의 경우 모든 손실은 (averaging을 통해) 단일 스칼라 값으로 결함됨  

### Objective function (Loss function):

- <font color="orange">**Choosing the right objective function for the right problem is extremely important**</font>:  
your network will take any shortcut it can, to minimize the loss; so if the objective doesn’t fully correlate with success for the task at hand, your network will end up doing things you may not have wanted.   
올바른 문제에 대한 올바른 목적 함수를 선택하는 것은 매우 중요함.
네트워크는 손실을 최소화하기 위해 가능한 모든 지름길을 택함. 따라서 목표가 당면한 작업의 성공과 완전히 연관되지 않으면 네트워크는 결국 원하지 않는 일을 하게 될 것임.
- <font color="orange">**Just remember that all neural networks you build will be just as ruthless in lowering their loss function**</font>   
—so choose the objective wisely, or you’ll have to face unintended side effects.  
여러분이 구축하는 모든 신경망은 손실 함수를 낮추는 데 있어 무자비할 것이라는 점을 기억해야함.   
— 그러니 목표를 현명하게 선택하지 않으면 의도하지 않은 부작용에 직면하게 될 것임.
- Fortunately, when it comes to common problems such as classification, regression, and sequence prediction, there are simple guidelines you can follow to choose the correct loss.  
다행히 분류, 회귀, 시퀀스 예측과 같은 일반적인 문제의 경우 올바른 손실을 선택하기 위해 따를 수 있는 간단한 지침이 있음
- For instance, you’ll use
  1. **Binary crossentropy** for a two-class classification problem,   
  Binary crossentropy: 두 클래스 분류 문제
  1. **Categorical crossentropy** for a many-class classification problem,  
  Categorical crossentropy: 다중클래스 분류 문제
  1. **Mean-squared error** for a regression problem,   
  Mean-squared error: 회귀 문제
  1. **Connectionist temporal classification (CTC)** for a sequence-learning problem, and so on.
  Connectionist temporal classification (CTC): 시퀀스 learning 문제
(참고: CTC는 입력 시퀀스의 길이와 출력 시퀀스의 길이가 다를 때, alignment 없이 학습과 예측을 가능하게 하는 손실 함수임)

- Only when you’re working on truly new research problems will you have to develop your own objective functions.   
정말로 새로운 연구 문제를 연구할 때만 자신만의 목적 함수를 개발해야 함  
- In the next few chapters, we’ll detail explicitly which loss functions to choose for a wide range of common tasks.

### How to define a model in Keras:

- There are two ways to define a model:
  - Using the **Sequential class** (only for linear stacks of layers, which is the most common network architecture by far)   
  Sequential 클래스 사용(가장 일반적인 네트워크 아키텍처인 선형 레이어 스택에만 해당)
  - The **functional API** (for directed acyclic graphs of layers, which lets you build completely arbitrary architectures).  
  함수형 API(레이어의 방향성 비순환 그래프를 위한 API로, 완전히 임의의 아키텍처를 구축할 수 있게 함)
  

### 1).  A two-layer model defined using the Sequential class:

Note that we’re passing the expected shape of the input data to the first layer

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(10, activation='softmax')) #마지막노드, 네트워크를 지정함에 있어서 시퀀셜한 층을 add, functinal api? 하단

In [ ]:
# sequential class 형태를 functional API 형태로 바꾸기
from keras import Model
from keras import layers, Input

# 입력 정의
inputs = Input(shape=(784,))

# 은닉층 및 출력층 연결
x = layers.Dense(32, activation='relu')(inputs) #fully connected layter, inputs 연결 후 x변수에 담아줌
outputs = layers.Dense(10, activation='softmax')(x) #10 node, x와 연결, output노드와 연결

# 모델 정의
model = Model(inputs=inputs, outputs=outputs)

#함수형 정의, sequential / API 형태, 함수형 지정이기 때문에 각각의 레이어를 좀 더 유연하게 구현이 가능하다.
# sequantial - add 로 구성
# functinal ..

### 2).  A two-layer model defined using the functional API:


In [ ]:
# 입력 정의
input_tensor = layers.Input(shape=(784,))

# 은닉층 및 출력층 연결
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# 모델 정의
model = models.Model(inputs=input_tensor, outputs=output_tensor)

#functinal API -> sequantial code
#바꿀 수 있어야 한다!!!!!!!!

```
# 코드로 형식 지정됨
```



In [ ]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(32, activation='relu', input_shape=(784,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))


In [ ]:
from keras import models
from keras import layers

# functional API 형태를 sequential class 형태로 바꾸기
# add 사용 하지 않고 하는 법
model = models.Sequential([
    layers.Dense(32, activation='relu', input_shape=(784,)),
    layers.Dense(64, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(10, activation='softmax')
])

With the functional API, you’re manipulating the data tensors that the model processes and applying layers to this tensor as if they were functions. (함수형 API를 사용하면 모델이 처리할 데이터 텐서를 만들고 마치 함수처럼 이 텐서에 층을 적용함)